# Recursive Style Seq2Seq Network

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell

In [2]:
NUM_INPUT_UNITS = 1
NUM_STATE_INPUT_UNITS = 11
NUM_HIDDEN_UNITS = 7
MAX_OUPUT_LEN = 3
MAX_RESCURISVE_DEPTH = 3
NUM_OUTPUTS = 2

In [131]:
tf.reset_default_graph()
inp_hidden = tf.placeholder(tf.float32, shape=(None, NUM_HIDDEN_UNITS))
inp_cell = tf.placeholder(tf.float32, shape=(None, NUM_HIDDEN_UNITS))
inp_states = (inp_hidden, inp_cell)

start_sentinel = tf.one_hot(
    tf.zeros(shape=(tf.shape(inp_hidden)[0],), dtype=tf.int32),
    NUM_OUTPUTS+1, dtype=tf.float32
)
rec_accum = tf.zeros_like(inp_hidden[:, 0])
rec_count = tf.cast(tf.zeros_like(inp_hidden[:, 0]), tf.int32)

cell = LSTMCell(NUM_HIDDEN_UNITS)
test_init = cell(start_sentinel, inp_states)

def network(prev_output, states):

    output, states = cell(prev_output, states)
    
    action_probs = tf.layers.dense(output, NUM_OUTPUTS, activation=tf.nn.softmax)
    
    return action_probs, output, states        

def cond(index, input_ta, output_ta, *states):
        return index <= MAX_OUPUT_LEN
    
def step(index, prev_out, output_ta, *states):
    
    # Get the current input
#     prev_out = input_ta.read(index)

    # Call the lstm cell
    action_probs, output, state_tuple = network(prev_out, states)
    states = state_tuple.h, state_tuple.c

    # out probs
    action_max = tf.argmax(action_probs, axis=1)

    # write the out actions one hot encoded to the next index
#     input_ta = input_ta.write(index+1, tf.one_hot(action_max, depth=NUM_OUTPUTS+1))
    action_max_one_hot = tf.one_hot(action_max, depth=NUM_OUTPUTS+1)

    # write the current action_prob output
    output_ta = output_ta.write(index, action_probs)

#     return (index+1, input_ta, output_ta) + tuple(states)
    return (index+1, action_max_one_hot, output_ta) + tuple(states)

def recursive_func(inp_hidden, inp_state, rec_accum, rec_count, output_ta_list=[]):
    """
    Takes in the input to a decoder, 
    """
    # Instantiate an index
    index = tf.constant(0, dtype=tf.int32)
    
    # Create an internal input_ta
#     input_ta = tf.TensorArray(tf.float32, size=MAX_OUPUT_LEN, dynamic_size=True, clear_after_read=False)
#     input_ta = input_ta.write(0, start_sentinel)
    
    # Create an internal output_ta
    output_ta = tf.TensorArray(tf.float32, size=MAX_RESCURISVE_DEPTH, dynamic_size=True)
    
    # while loop
    
    final_index, final_action, output_ta_final, _, _ = tf.while_loop(
        cond,
        step,
        loop_vars=[index, start_sentinel, output_ta, inp_hidden, inp_state]
    )
    
    output_ta_list.append(output_ta_final)
    
#     if rec_count <= 0:
#         return
    
    def true_fn():
        return recursive_func(inp_hidden, inp_state, rec_accum, rec_count - 1, output_ta_list=output_ta_list)
    
    def false_fn():
        return []
    
    tf.cond(
        tf.greater(rec_count, 0),
        true_fn=true_fn,
        false_fn=false_fn
    )
    
#     recursive_func(inp_hidden, inp_state, rec_accum, rec_count - 1, output_ta_list=output_ta_list)
    
    return final_index, final_action, output_ta_final
#     return res[0], res[1], res[2]

In [132]:
output_list = []
final_index, final_action, output_ta_final = recursive_func(inp_hidden, inp_cell, 0, 2, output_ta_list=output_list)

recursive count is 2
recursive count is 1
recursive count is 0
recursive count is -1
recursive count is -2
recursive count is -3
recursive count is -4
recursive count is -5
recursive count is -6
recursive count is -7
recursive count is -8
recursive count is -9
recursive count is -10
recursive count is -11
recursive count is -12
recursive count is -13
recursive count is -14
recursive count is -15
recursive count is -16
recursive count is -17
recursive count is -18
recursive count is -19
recursive count is -20
recursive count is -21
recursive count is -22
recursive count is -23
recursive count is -24
recursive count is -25
recursive count is -26
recursive count is -27
recursive count is -28
recursive count is -29
recursive count is -30
recursive count is -31
recursive count is -32
recursive count is -33
recursive count is -34
recursive count is -35
recursive count is -36
recursive count is -37
recursive count is -38
recursive count is -39
recursive count is -40
recursive count is -41
rec

KeyboardInterrupt: 

In [92]:
output_list_mat = map(lambda ta: ta.stack(), output_list)

In [93]:
sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

In [75]:
res = sess.run(
    [rec_accum, rec_count, start_sentinel, mat],
    feed_dict={
        inp_hidden: np.random.rand(10, NUM_HIDDEN_UNITS),
        inp_cell: np.random.rand(10, NUM_HIDDEN_UNITS)
    }
)

ValueError: Fetch argument <tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 2) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("TensorArrayStack/TensorArrayGatherV3:0", shape=(?, ?, 2), dtype=float32) is not an element of this graph.)

In [121]:
res = sess.run(
    output_list_mat,
    feed_dict={
        inp_hidden: np.random.rand(2, NUM_HIDDEN_UNITS),
        inp_cell: np.random.rand(2, NUM_HIDDEN_UNITS)
    }
)

In [122]:
print np.argmax(res[0], axis=-1)
print
print np.argmax(res[1], axis=-1)
print
print np.argmax(res[2], axis=-1)

[[1 0]
 [1 1]
 [1 1]
 [1 1]]

[[1 1]
 [0 0]
 [0 0]
 [0 0]]

[[0 1]
 [0 0]
 [0 0]
 [0 0]]
